In [9]:
from google.colab import files
uploaded = files.upload()

Saving dataLabeled.csv to dataLabeled (1).csv


In [16]:
import pandas as pd
import io
import spacy
from gensim.models.word2vec import Word2Vec
import re, string
from nltk.corpus import stopwords

stopword = stopwords.words('english')

df = pd.read_csv(io.StringIO(uploaded['dataLabeled.csv'].decode('utf-8')))
df


,X,Title,Company,Location,Type,Salary,Contract_type,Job_Description,JobType,Industry,experience,exp_normailzed,type_norm,Contract_type_norm,salary1,salary_min,salary_max,salary_avg,salary_norm,label
0,10,Investment Banking Associate,Q Advisors,"Denver, CO 80202",Hybrid remote,"$140,000 - $175,000",Full-time,Overview:\nQ Advisors is actively seeking an A...,Banking,Finance,1 year,0.1,-2.365042,1,"$140,000 - $175,000",140000.0,175000.0,157500.0,157500.0,High
1,20,Associate Project Finance Investment Banking,MUFG,"New York, NY 10020",Onsite,"$150,000 - $175,000",Full-time,Do you want your voice heard and your actions ...,Banking,Finance,3 years,0.3,-0.063127,1,"$150,000 - $175,000",150000.0,175000.0,162500.0,162500.0,High
2,22,Senior Consultant - AML Transaction Monitoring...,Vitae Advisors,Remote,Onsite,$30 - $42,Contract,We are actively supporting a team to review th...,Banking,Finance,4+ years,0.4,-0.063127,0,$30 - $42,30.0,42.0,36.0,74880.0,Medium
3,25,Senior Relationship Banker,Turn2Partners,Remote,Onsite,$24 - $29,Contract,We are working on a contract-to-hire Senior Re...,Banking,Finance,2+ years,0.2,-0.063127,0,$24 - $29,24.0,29.0,26.5,55120.0,Medium
4,31,"Analyst, Global Corporate Investment Banking -...",MUFG,"Chicago, IL 60606",Onsite,"$90,000 - $110,000",Full-time,Do you want your voice heard and your actions ...,Banking,Finance,3 years,0.3,-0.063127,1,"$90,000 - $110,000",90000.0,110000.0,100000.0,100000.0,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,300,Senior Data Analytics Engineer,INPOSIA Solutions GmbH,Remote,Onsite,"$157,300 - $259,500",Contract,Job Description\nOverview\nWe are building clo...,System Embedded Engineer,Electrical Engineering,5 years,0.5,-0.063127,0,"$157,300 - $259,500",157300.0,259500.0,208400.0,208400.0,High
1081,302,Power System Automation Specialist,Schweitzer Engineering Laboratories,"Houston, TX 77072",Onsite,"$73,150 - $114,400",Full-time,The devices we make at SEL are embedded in the...,System Embedded Engineer,Electrical Engineering,4+ years,0.4,-0.063127,1,"$73,150 - $114,400",73150.0,114400.0,93775.0,93775.0,Medium
1082,303,Firmware Engineer,Gradient Comfort,"San Francisco, CA",Remote,"$110,000 - $185,000",Contract,Gradient's mission is to cool the world by tra...,System Embedded Engineer,Electrical Engineering,30 years,3.0,2.238789,0,"$110,000 - $185,000",110000.0,185000.0,147500.0,147500.0,High
1083,304,Embedded Software Engineer,Venture Connections,"Warner Robins, GA",Onsite,"$82,000 - $140,000",Full-time,Seeking a highly motivated and creative Embedd...,System Embedded Engineer,Electrical Engineering,5 years,0.5,-0.063127,1,"$82,000 - $140,000",82000.0,140000.0,111000.0,111000.0,High


In [17]:
def clean(text):
    text = text.lower() # lower case
    text = re.sub(r'https?://\S+|www\.\S+', " ", text)#URL
    text = re.sub(r'@\w+',' ',text) # mentions
    text = re.sub(r'#\w+', ' ', text) #hashtags
    text= re.sub(r'[^\w\s]',' ',text) #punctuations
    text= re.sub(r'\d+', ' ', text) #digits
    text = re.sub('\[.*?\]',' ', text) #any punctuations left
    text = re.sub("[^a-z0-9]"," ", text)# any others charcters other than numbers and letters
    text = text.split() #stopwords
    text = " ".join([word for word in text if not word in stopword])
    return text

df['Job_Description'] = df['Job_Description'].astype(str).apply(lambda x: clean(x))

In [18]:
df.head(10)

,X,Title,Company,Location,Type,Salary,Contract_type,Job_Description,JobType,Industry,experience,exp_normailzed,type_norm,Contract_type_norm,salary1,salary_min,salary_max,salary_avg,salary_norm,label
0,10,Investment Banking Associate,Q Advisors,"Denver, CO 80202",Hybrid remote,"$140,000 - $175,000",Full-time,overview q advisors actively seeking associate...,Banking,Finance,1 year,0.1,-2.365042,1,"$140,000 - $175,000",140000.0,175000.0,157500.0,157500.0,High
1,20,Associate Project Finance Investment Banking,MUFG,"New York, NY 10020",Onsite,"$150,000 - $175,000",Full-time,want voice heard actions count discover opport...,Banking,Finance,3 years,0.3,-0.063127,1,"$150,000 - $175,000",150000.0,175000.0,162500.0,162500.0,High
2,22,Senior Consultant - AML Transaction Monitoring...,Vitae Advisors,Remote,Onsite,$30 - $42,Contract,actively supporting team review thousands corr...,Banking,Finance,4+ years,0.4,-0.063127,0,$30 - $42,30.0,42.0,36.0,74880.0,Medium
3,25,Senior Relationship Banker,Turn2Partners,Remote,Onsite,$24 - $29,Contract,working contract hire senior relationship bank...,Banking,Finance,2+ years,0.2,-0.063127,0,$24 - $29,24.0,29.0,26.5,55120.0,Medium
4,31,"Analyst, Global Corporate Investment Banking -...",MUFG,"Chicago, IL 60606",Onsite,"$90,000 - $110,000",Full-time,want voice heard actions count discover opport...,Banking,Finance,3 years,0.3,-0.063127,1,"$90,000 - $110,000",90000.0,110000.0,100000.0,100000.0,Medium
5,32,Senior Consultant - AML Transaction Monitoring...,Vitae Advisors,Remote,Onsite,$30 - $42,Contract,actively supporting team review thousands corr...,Banking,Finance,4+ years,0.4,-0.063127,0,$30 - $42,30.0,42.0,36.0,74880.0,Medium
6,38,Investment Banking Analyst,Citi,"New York, NY",Onsite,"$110,000 - $135,000",Full-time,investment banking analyst intermediate level ...,Banking,Finance,1-3 years,1.3,-0.063127,1,"$110,000 - $135,000",110000.0,135000.0,122500.0,122500.0,High
7,46,Private Banking Relationship Manager III,Huntington Bank,"Centennial, CO 80112",Onsite,"$125,000 - $245,000",Full-time,description summary private banking relationsh...,Banking,Finance,4 years,0.4,-0.063127,1,"$125,000 - $245,000",125000.0,245000.0,185000.0,185000.0,High
8,48,Personal Banker - Laurel Lakes Branch (Resident),Citi,"Laurel, MD 20707",Onsite,"$56,300 - $78,550",Full-time,personal banker safe act entry level position ...,Banking,Finance,0-2 years,0.2,-0.063127,1,"$56,300 - $78,550",56300.0,78550.0,67425.0,67425.0,Medium
9,51,Investment Banking Associate - Financial Services,Truist Financial,"New York, NY 10036",Onsite,"$175,000 - $225,000",Full-time,position described want apply click apply butt...,Banking,Finance,4 years,0.4,-0.063127,1,"$175,000 - $225,000",175000.0,225000.0,200000.0,200000.0,High


In [19]:
import spacy

# spacy.load is a method in the Spacy library for loading pre-trained models for natural language processing.
# load english language model and create nlp object from it
# Pre-trained English language model called en_core_web_sm. 
# This model is a small and fast model for English language text processing, and it includes features such as part-of-speech tagging, named entity recognition, and dependency parsing
nlp = spacy.load("en_core_web_sm") 

def preprocess(text):
    # remove stop words and lemmatize the text

    #For converting paragrapg to sentence
    # spacy. is object oriented
    # Dr. Ram is eating.He is not available.
    # -> [Dr. Ram is eating. He is not available]
    doc = nlp(text)

    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    
    return " ".join(filtered_tokens) 

In [20]:
df['preprocessed_txt'] = df['Job_Description'].apply(preprocess) 


In [21]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    df.preprocessed_txt, 
    df.label, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022, #keep using the same sample for training or testing
    stratify=df.label #balance category used in training the model
)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer


In [23]:
vectorizer = CountVectorizer(ngram_range= (1,3)).fit(x_train)
train_X_cv = vectorizer.transform(x_train)
test_X_cv = vectorizer.transform(x_test)

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [25]:
lr = LogisticRegression(penalty="l2",multi_class = "multinomial",solver = "saga", C=1, max_iter=5000)
lr.fit(train_X_cv,y_train)
lr_pred = lr.predict(test_X_cv)
print(classification_report(y_test, lr_pred))

              precision    recall  f1-score   support

        High       0.89      0.92      0.91       145
         Low       0.00      0.00      0.00         4
      Medium       0.80      0.78      0.79        68

    accuracy                           0.86       217
   macro avg       0.56      0.57      0.57       217
weighted avg       0.84      0.86      0.85       217



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
